In [13]:
import os
import shutil
import certifi
import tempfile

caminho_ruim_cert = certifi.where()
print(f"Original (com erro): {caminho_ruim_cert}")

with tempfile.NamedTemporaryFile(delete=False, suffix='.pem') as tmp_file:
    with open(caminho_ruim_cert, 'rb') as source_file:
        shutil.copyfileobj(source_file, tmp_file)
    caminho_seguro_cert = tmp_file.name

# Se estiver atrás de proxy corporativo (MITM), adicione o CA corporativo aqui (em PEM).
# Ex.: defina a env var EXTRA_CA_PEM=c:\caminho\corporate_ca.pem
extra_ca_path = os.environ.get("EXTRA_CA_PEM")
if extra_ca_path and os.path.exists(extra_ca_path):
    with open(extra_ca_path, "rb") as extra_in, open(caminho_seguro_cert, "ab") as out:
        out.write(b"\n")
        out.write(extra_in.read())
    print(f"CA extra anexado ao bundle: {extra_ca_path}")

print(f"Novo caminho seguro criado: {caminho_seguro_cert}")

# Requests
os.environ['CURL_CA_BUNDLE'] = caminho_seguro_cert
os.environ['SSL_CERT_FILE'] = caminho_seguro_cert
os.environ['REQUESTS_CA_BUNDLE'] = caminho_seguro_cert

import yfinance as yf
import polars as pl
import polars.selectors as cs
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

Original (com erro): c:\Users\william.mendes\OneDrive\Área de Trabalho\projects\pos\tech_challenge_2_mlet\.venv\Lib\site-packages\certifi\cacert.pem
Novo caminho seguro criado: C:\Users\WILLIA~1.MEN\AppData\Local\Temp\tmplup0_g_w.pem


In [18]:

tickers_all = ['PETR4.SA', 'VALE3.SA', 'ITUB4.SA', 'BBDC4.SA']
# Para debug, use apenas 1 ticker:
tickers = ['PETR4.SA']
# Para rodar tudo, troque para: tickers = tickers_all

In [19]:
today = datetime.now()

# IMPORTANTE (yfinance): `start` é inclusivo e `end` é EXCLUSIVO.
# Se você quer incluir D-1 (ontem), passe `end=today`.
end_date_obj = today  # exclusivo
end_date = end_date_obj.strftime('%Y-%m-%d')

end_inclusive_obj = today - timedelta(days=1)  # só para exibir no print

# Dados dos últimos 1 mês (janela inclusiva)
start_date_obj = end_inclusive_obj - relativedelta(months=1)
start_date = start_date_obj.strftime('%Y-%m-%d')

print(f"Executando pipeline para janela: {start_date} até {end_inclusive_obj.strftime('%Y-%m-%d')} (end exclusivo: {end_date})")

Executando pipeline para janela: 2025-12-12 até 2026-01-12 (end exclusivo: 2026-01-13)


In [20]:
import time
from curl_cffi import requests as crequests

data_list = []

# yfinance 1.0 usa curl_cffi por baixo; se passar `session`, precisa ser curl_cffi.requests.Session
session = crequests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36"
})

# Importante: o erro que você estava vendo vem do SSL do libcurl (curl_cffi).
# Se você está atrás de proxy corporativo, talvez precise adicionar o CA corporativo ao bundle.
VERIFY_SSL = True
session.verify = caminho_seguro_cert if VERIFY_SSL else False

print("\n>>> Iniciando Download das ações")

for ticker in tickers:
    for attempt in (1, 2):
        df_y = None
        try:
            df_y = yf.download(
                ticker,
                start=start_date,
                end=end_date,  # exclusivo
                interval="1d",
                actions=False,
                auto_adjust=False,
                repair=True,
                progress=False,
                threads=False,
                timeout=20,
                session=session,
            )
        except Exception as e:
            if attempt == 1:
                time.sleep(1.0)
                continue
            print(f"Erro {ticker}: {type(e).__name__}: {e}")
            df_y = None

        if df_y is None or df_y.empty:
            if attempt == 1:
                time.sleep(1.0)
                continue

            print(f"Vazio/erro ao baixar: {ticker}")
            # Diagnóstico rápido (ajuda a detectar 401/403/HTML de proxy/captcha/SSL)
            try:
                url = f"http://query1.finance.yahoo.com/v8/finance/chart/{ticker}?range=1mo&interval=1d"
                r = session.get(url, timeout=20)
                print("Diagnóstico HTTP:", r.status_code, r.headers.get("content-type"))
                print("Body (inicio):", (r.text or "")[:200])
            except Exception as diag_e:
                print("Falha no diagnóstico HTTP:", type(diag_e).__name__, str(diag_e))
            break

        df_y = df_y.reset_index()
        if "Datetime" in df_y.columns and "Date" not in df_y.columns:
            df_y = df_y.rename(columns={"Datetime": "Date"})
        df_y["Ticker"] = ticker

        pl_df = pl.from_pandas(df_y)
        data_list.append(pl_df)
        print(f"Sucesso: {ticker} ({pl_df.shape[0]} linhas)")
        break

if not data_list:
    raise RuntimeError("Nenhum ticker foi baixado; verifique o diagnóstico acima. Se for SSL, teste VERIFY_SSL=False só para confirmar a causa.")

df_raw = pl.concat(data_list).sort(["Ticker", "Date"])
print(f"\nTotal de linhas baixadas: {df_raw.shape[0]}")
print(df_raw.head())

Failed to get ticker 'PETR4.SA' reason: Failed to perform, curl: (60) SSL certificate problem: unable to get local issuer certificate. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
$PETR4.SA: possibly delisted; no timezone found

1 Failed download:
['PETR4.SA']: possibly delisted; no timezone found



>>> Iniciando Download das ações


Failed to get ticker 'PETR4.SA' reason: Failed to perform, curl: (60) SSL certificate problem: unable to get local issuer certificate. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
$PETR4.SA: possibly delisted; no timezone found

1 Failed download:
['PETR4.SA']: possibly delisted; no timezone found


Vazio/erro ao baixar: PETR4.SA
Falha no diagnóstico HTTP: CertificateVerifyError Failed to perform, curl: (60) SSL certificate problem: unable to get local issuer certificate. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.


RuntimeError: Nenhum ticker foi baixado; verifique o diagnóstico acima. Se for SSL, teste VERIFY_SSL=False só para confirmar a causa.

In [ ]:
# Transformação (Feature Engineering)
if "df_raw" not in globals():
    raise RuntimeError("df_raw não existe. Rode a célula de download e resolva os erros antes de transformar.")

df_refined = df_raw.with_columns([
    # Cast de data
    pl.col("Date").cast(pl.Date).alias("data_pregao"),

    # Lowercase
    pl.col("Ticker").str.to_lowercase().alias("nome_acao"),

    # Renomeando colunas
    pl.col("Open").alias("abertura"),
    pl.col("Close").alias("fechamento"),
    pl.col("High").alias("max"),
    pl.col("Low").alias("min"),
    pl.col("Volume").alias("volume_negociado"),
    
    # Média Móvel de 7 dias agrupado por Ticker
    pl.col("Close")
        .rolling_mean(window_size=7)
        .over("Ticker")
        .alias("media_movel_7d"),

    # Lag 1: O valor de fechamento do dia anterior agrupado por Ticker
    pl.col("Close")
        .shift(1)
        .over("Ticker")
        .alias("lag_1d"),
        
    # Lag 2: O valor de fechamento de 2 dias atrás agrupado por Ticker
    pl.col("Close")
        .shift(2)
        .over("Ticker")
        .alias("lag_2d"),
    
    # Lag 3: O valor de fechamento de 3 dias atrás agrupado por Ticker
    pl.col("Close")
        .shift(3)
        .over("Ticker")
        .alias("lag_3d")
])

# Remove linhas com valores nulos
df_final = df_refined.drop_nulls()

# Arredonda valores float para 2 casas decimais
df_final = df_final.with_columns(cs.float().round(2))

# Remove colunas desnecessárias
df_final = df_final.select([
    "data_pregao", "nome_acao", "abertura", "fechamento", "max", "min",
    "volume_negociado", "media_movel_7d", "lag_1d", "lag_2d", "lag_3d"
])

NameError: name 'df_raw' is not defined

In [ ]:
print(df_final.head(10))


--- Amostra com Lags e Média Móvel ---
shape: (10, 11)
┌─────────────┬───────────┬──────────┬────────────┬───┬────────────────┬────────┬────────┬────────┐
│ data_pregao ┆ nome_acao ┆ abertura ┆ fechamento ┆ … ┆ media_movel_7d ┆ lag_1d ┆ lag_2d ┆ lag_3d │
│ ---         ┆ ---       ┆ ---      ┆ ---        ┆   ┆ ---            ┆ ---    ┆ ---    ┆ ---    │
│ date        ┆ str       ┆ f64      ┆ f64        ┆   ┆ f64            ┆ f64    ┆ f64    ┆ f64    │
╞═════════════╪═══════════╪══════════╪════════════╪═══╪════════════════╪════════╪════════╪════════╡
│ 2025-07-18  ┆ intb3.sa  ┆ 13.76    ┆ 13.35      ┆ … ┆ 13.47          ┆ 13.88  ┆ 13.79  ┆ 13.78  │
│ 2025-07-21  ┆ intb3.sa  ┆ 13.4     ┆ 13.62      ┆ … ┆ 13.54          ┆ 13.35  ┆ 13.88  ┆ 13.79  │
│ 2025-07-22  ┆ intb3.sa  ┆ 13.61    ┆ 13.35      ┆ … ┆ 13.57          ┆ 13.62  ┆ 13.35  ┆ 13.88  │
│ 2025-07-23  ┆ intb3.sa  ┆ 13.35    ┆ 13.72      ┆ … ┆ 13.64          ┆ 13.35  ┆ 13.62  ┆ 13.35  │
│ 2025-07-24  ┆ intb3.sa  ┆ 13.72    ┆ 13.65

In [ ]:
# Agrupamento/Sumarização (mensal)
df_agregado = (
    df_final
    .with_columns(pl.col("data_pregao").dt.truncate("1mo").alias("mes"))
    .group_by(["nome_acao", "mes"])
    .agg([
        pl.col("volume_negociado").sum().alias("volume_mensal_total"),
        pl.col("fechamento").mean().alias("preco_medio_mensal").round(2),
    ])
    .sort(["nome_acao", "mes"])
 )

print("\n--- Agregação Mensal ---")
print(df_agregado.head())


--- Agregação Mensal ---
shape: (5, 4)
┌──────────┬─────────────┬─────────────────────┬────────────────────┐
│ Ticker   ┆ data_pregao ┆ volume_mensal_total ┆ preco_medio_mensal │
│ ---      ┆ ---         ┆ ---                 ┆ ---                │
│ str      ┆ date        ┆ i64                 ┆ f64                │
╞══════════╪═════════════╪═════════════════════╪════════════════════╡
│ INTB3.SA ┆ 2025-07-01  ┆ 32258900            ┆ 13.54              │
│ INTB3.SA ┆ 2025-08-01  ┆ 41764600            ┆ 11.8               │
│ INTB3.SA ┆ 2025-09-01  ┆ 49118000            ┆ 11.68              │
│ INTB3.SA ┆ 2025-10-01  ┆ 52991000            ┆ 10.65              │
│ INTB3.SA ┆ 2025-11-01  ┆ 38471300            ┆ 11.7               │
└──────────┴─────────────┴─────────────────────┴────────────────────┘


In [33]:
# Exemplo de salvamento local em Parquet (como seria feito no S3)
# df_refined.write_parquet("dados_refined.parquet")